In [1]:
!ls

gold_recovery_full_new.csv  gold_recovery_train_new.csv
gold_recovery_test_new.csv  ntb.ipynb


# 1. Подготовьте данные
## 1.1. Откройте файлы и изучите их.

In [2]:
import pandas as pd 
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

from sklearn.metrics import mean_absolute_error


In [3]:
train_df = pd.read_csv('gold_recovery_train_new.csv')
test_df = pd.read_csv('gold_recovery_test_new.csv')
full_df = pd.read_csv('gold_recovery_full_new.csv')

In [4]:
dfs = [train_df, test_df, full_df]

## 1.2. Проверьте, что эффективность обогащения рассчитана правильно. Вычислите её на обучающей выборке для признака rougher.output.recovery. Найдите MAE между вашими расчётами и значением признака. Опишите выводы.

In [5]:
train_df['rougher.output.recovery'] #то что перепроверям 

0        87.107763
1        86.843261
2        86.842308
3        87.226430
4        86.688794
           ...    
14144    89.574376
14145    87.724007
14146    88.890579
14147    89.858126
14148    89.514960
Name: rougher.output.recovery, Length: 14149, dtype: float64

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14149 entries, 0 to 14148
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                14149 non-null  object 
 1   final.output.concentrate_ag                         14148 non-null  float64
 2   final.output.concentrate_pb                         14148 non-null  float64
 3   final.output.concentrate_sol                        13938 non-null  float64
 4   final.output.concentrate_au                         14149 non-null  float64
 5   final.output.recovery                               14149 non-null  float64
 6   final.output.tail_ag                                14149 non-null  float64
 7   final.output.tail_pb                                14049 non-null  float64
 8   final.output.tail_sol                               14144 non-null  float64


In [7]:
#display(train_df.iloc[:, 30:65].head())
display(train_df.iloc[:, 30:65].columns)

Index(['rougher.calculation.sulfate_to_au_concentrate',
       'rougher.calculation.floatbank10_sulfate_to_au_feed',
       'rougher.calculation.floatbank11_sulfate_to_au_feed',
       'rougher.calculation.au_pb_ratio', 'rougher.input.feed_ag',
       'rougher.input.feed_pb', 'rougher.input.feed_rate',
       'rougher.input.feed_size', 'rougher.input.feed_sol',
       'rougher.input.feed_au', 'rougher.input.floatbank10_sulfate',
       'rougher.input.floatbank10_xanthate',
       'rougher.input.floatbank11_sulfate',
       'rougher.input.floatbank11_xanthate', 'rougher.output.concentrate_ag',
       'rougher.output.concentrate_pb', 'rougher.output.concentrate_sol',
       'rougher.output.concentrate_au', 'rougher.output.recovery',
       'rougher.output.tail_ag', 'rougher.output.tail_pb',
       'rougher.output.tail_sol', 'rougher.output.tail_au',
       'rougher.state.floatbank10_a_air', 'rougher.state.floatbank10_a_level',
       'rougher.state.floatbank10_b_air', 'rougher.state.floa

сначала нужно посчитть это: 

- C — доля золота в концентрате после флотации/очистки; (+) (rougher.output.feed_au_share)
- F — доля золота в сырье/концентрате до флотации/очистки; (+) (rougher.input.feed_au_share)
- T — доля золота в отвальных хвостах после флотации/очистки.

In [8]:
#- C — доля золота в концентрате после флотации/очистки
train_df.iloc[:, 43:53].head().columns  

volume_sum_columns = ['rougher.output.concentrate_ag', 'rougher.output.concentrate_pb',
                       'rougher.output.concentrate_sol', 'rougher.output.concentrate_au']

train_df['rougher.output.feed_sum'] = train_df[volume_sum_columns].sum(axis =1)

train_df['rougher.output.feed_au_share'] = train_df['rougher.output.concentrate_au']/train_df['rougher.output.feed_sum']

C = train_df['rougher.output.feed_au_share']

In [9]:
#колонки инпута
#- F — доля золота в сырье/концентрате до флотации/очистки;
train_df.iloc[:, 34:44].head().columns

volume_sum_columns = ['rougher.input.feed_ag', 'rougher.input.feed_pb', 
                      'rougher.input.feed_sol', 'rougher.input.feed_au']

train_df['rougher.input.feed_sum'] = train_df[volume_sum_columns].sum(axis =1)

train_df['rougher.input.feed_au_share'] = train_df['rougher.input.feed_au']/train_df['rougher.input.feed_sum']

F = train_df['rougher.input.feed_au_share']

In [10]:
#- T — доля золота в отвальных хвостах после флотации/очистки.
train_df.iloc[:, 49:53].head().columns 

volume_sum_columns = ['rougher.output.tail_ag', 'rougher.output.tail_pb', 
                      'rougher.output.tail_sol', 'rougher.output.tail_au']

train_df['rougher.output.tail_feed_sum'] = train_df[volume_sum_columns].sum(axis =1)

train_df['rougher.output.tail_feed_au_share'] = train_df['rougher.output.tail_au']/train_df['rougher.output.tail_feed_sum']

T = train_df['rougher.output.tail_feed_au_share'] 

In [11]:
#recovery test
train_df['recovery_test'] = (C*(F-T))/(F*(C-T))*100
train_df[['recovery_test', 'rougher.output.recovery']].isna().sum()

recovery_test              297
rougher.output.recovery      0
dtype: int64

In [12]:
#появились наны из-за нулей в rougher.output.concentrate_so
train_df[train_df['recovery_test'].isnull()]

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.output.concentrate_ag,primary_cleaner.output.concentrate_pb,primary_cleaner.output.concentrate_sol,primary_cleaner.output.concentrate_au,primary_cleaner.output.tail_ag,primary_cleaner.output.tail_pb,primary_cleaner.output.tail_sol,primary_cleaner.output.tail_au,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.calculation.sulfate_to_au_concentrate,rougher.calculation.floatbank10_sulfate_to_au_feed,rougher.calculation.floatbank11_sulfate_to_au_feed,rougher.calculation.au_pb_ratio,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,rougher.input.floatbank10_xanthate,rougher.input.floatbank11_sulfate,rougher.input.floatbank11_xanthate,rougher.output.concentrate_ag,rougher.output.concentrate_pb,rougher.output.concentrate_sol,rougher.output.concentrate_au,rougher.output.recovery,rougher.output.tail_ag,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_au,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.output.tail_ag,secondary_cleaner.output.tail_pb,secondary_cleaner.output.tail_sol,secondary_cleaner.output.tail_au,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level,rougher.output.feed_sum,rougher.output.feed_au_share,rougher.input.feed_sum,rougher.input.feed_au_share,rougher.output.tail_feed_sum,rougher.output.tail_feed_au_share,recovery_test
45,2016-01-16 21:00:00,4.566664,10.406178,4.588698,46.614456,68.403877,10.705777,1.509070,17.875134,2.509776,131.251366,13.969457,7.25,1.203034,8.513177,9.724797,19.120964,37.633572,16.455383,3.366501,6.864404,3.434734,1601.674776,-500.285160,1600.151967,-499.633542,1599.542959,-499.495721,1599.342687,-500.360306,41359.061159,3196.130244,3205.717062,2.530283,7.521974,2.811569,581.605260,65.620714,41.154430,7.114065,13.322740,7.504592,13.311361,7.493738,0.0,0.0,0.0,0.0,0.0,4.784398,0.818965,20.046797,1.939811,999.026619,-499.850895,1600.614880,-500.331775,1598.654561,-500.364377,1398.394100,-501.540155,1197.182011,-500.205806,1198.542180,-499.648335,13.605154,4.983564,9.545239,3.102137,25.963860,-498.097243,23.896954,-500.047419,24.963926,-500.207267,23.984937,-500.613261,14.025588,-550.404242,12.028925,-550.570104,10.052072,-500.797905,7.984500,-499.581046,11.982025,-599.161228,

In [13]:
#посмотрим что без нанов 
print(mean_absolute_error(train_df.dropna(subset=['recovery_test'])['rougher.output.recovery'], 
                            train_df.dropna(subset=['recovery_test'])['recovery_test']))

18.814459214503362


## 1.3. Проанализируйте признаки, недоступные в тестовой выборке. Что это за параметры? К какому типу относятся?

In [14]:
test_df.isnull().sum().sort_values(ascending=False)

rougher.input.floatbank11_xanthate            25
rougher.input.feed_sol                        21
secondary_cleaner.state.floatbank3_a_air       9
rougher.input.floatbank11_sulfate              8
primary_cleaner.input.depressant               5
rougher.input.floatbank10_sulfate              5
primary_cleaner.input.xanthate                 4
primary_cleaner.input.sulfate                  4
rougher.input.feed_rate                        3
secondary_cleaner.state.floatbank2_a_air       3
secondary_cleaner.state.floatbank2_b_air       2
rougher.input.feed_size                        1
secondary_cleaner.state.floatbank2_b_level     0
rougher.state.floatbank10_f_level              0
rougher.state.floatbank10_f_air                0
secondary_cleaner.state.floatbank3_a_level     0
secondary_cleaner.state.floatbank2_a_level     0
date                                           0
rougher.state.floatbank10_e_air                0
secondary_cleaner.state.floatbank3_b_air       0
secondary_cleaner.st

In [15]:
print('columns in train', len(train_df.columns))
print('columns in test', len(test_df.columns))

print(len(set(train_df.columns).difference(set(test_df.columns))))
train_test_dif_cols = set(train_df.columns).difference(set(test_df.columns))

columns in train 94
columns in test 53
41


In [16]:
list(train_test_dif_cols)

['primary_cleaner.output.concentrate_sol',
 'rougher.output.concentrate_au',
 'primary_cleaner.output.concentrate_pb',
 'rougher.output.feed_au_share',
 'primary_cleaner.output.tail_sol',
 'final.output.tail_pb',
 'secondary_cleaner.output.tail_pb',
 'primary_cleaner.output.tail_ag',
 'rougher.output.tail_feed_sum',
 'final.output.recovery',
 'secondary_cleaner.output.tail_ag',
 'rougher.input.feed_sum',
 'rougher.output.tail_pb',
 'rougher.output.tail_au',
 'primary_cleaner.output.concentrate_au',
 'rougher.output.tail_feed_au_share',
 'rougher.calculation.floatbank11_sulfate_to_au_feed',
 'secondary_cleaner.output.tail_au',
 'recovery_test',
 'rougher.output.tail_sol',
 'final.output.tail_ag',
 'final.output.concentrate_sol',
 'secondary_cleaner.output.tail_sol',
 'primary_cleaner.output.concentrate_ag',
 'final.output.concentrate_au',
 'primary_cleaner.output.tail_pb',
 'rougher.calculation.sulfate_to_au_concentrate',
 'final.output.concentrate_pb',
 'final.output.tail_sol',
 'rough

## 1.4. Проведите предобработку данных.

Что можно сделать в предобработке: 

1. удалить из train колонки, которые недоступны в test 
2. Удалить пропуски, если их немного 
3. сбросить дубликаты 

In [17]:
cols = test_df.columns

for df in dfs: 
    #проверка на дубликаты 
    print(len(df.columns))
    #тут пока непонятно, стоит ли до EDA удалять колонки 
    #df = df[cols]
    print(len(df.columns))
    print(df.shape)
    print(df.duplicated().sum())
    print(df.isnull().sum().sort_values(ascending=False))
    

94
94
(14149, 94)
0
secondary_cleaner.output.tail_sol                     1605
rougher.state.floatbank10_e_air                        436
rougher.input.floatbank11_xanthate                     428
rougher.output.feed_au_share                           297
recovery_test                                          297
primary_cleaner.output.concentrate_sol                 286
secondary_cleaner.state.floatbank2_a_air               217
final.output.concentrate_sol                           211
rougher.input.feed_size                                144
final.output.tail_pb                                   100
rougher.input.feed_pb                                  100
primary_cleaner.input.xanthate                         100
primary_cleaner.output.concentrate_pb                   86
rougher.input.feed_sol                                  78
primary_cleaner.output.tail_sol                         46
rougher.input.floatbank11_sulfate                       36
primary_cleaner.input.depressant    

# 2. Проанализируйте данные


## 2.1. Посмотрите, как меняется концентрация металлов (Au, Ag, Pb) на различных этапах очистки. Опишите выводы.

### rougher

In [18]:
full_df.head()

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.output.concentrate_ag,primary_cleaner.output.concentrate_pb,primary_cleaner.output.concentrate_sol,primary_cleaner.output.concentrate_au,primary_cleaner.output.tail_ag,primary_cleaner.output.tail_pb,primary_cleaner.output.tail_sol,primary_cleaner.output.tail_au,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,primary_cleaner.state.floatbank8_c_level,primary_cleaner.state.floatbank8_d_air,primary_cleaner.state.floatbank8_d_level,rougher.calculation.sulfate_to_au_concentrate,rougher.calculation.floatbank10_sulfate_to_au_feed,rougher.calculation.floatbank11_sulfate_to_au_feed,rougher.calculation.au_pb_ratio,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,rougher.input.floatbank10_xanthate,rougher.input.floatbank11_sulfate,rougher.input.floatbank11_xanthate,rougher.output.concentrate_ag,rougher.output.concentrate_pb,rougher.output.concentrate_sol,rougher.output.concentrate_au,rougher.output.recovery,rougher.output.tail_ag,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_au,rougher.state.floatbank10_a_air,rougher.state.floatbank10_a_level,rougher.state.floatbank10_b_air,rougher.state.floatbank10_b_level,rougher.state.floatbank10_c_air,rougher.state.floatbank10_c_level,rougher.state.floatbank10_d_air,rougher.state.floatbank10_d_level,rougher.state.floatbank10_e_air,rougher.state.floatbank10_e_level,rougher.state.floatbank10_f_air,rougher.state.floatbank10_f_level,secondary_cleaner.output.tail_ag,secondary_cleaner.output.tail_pb,secondary_cleaner.output.tail_sol,secondary_cleaner.output.tail_au,secondary_cleaner.state.floatbank2_a_air,secondary_cleaner.state.floatbank2_a_level,secondary_cleaner.state.floatbank2_b_air,secondary_cleaner.state.floatbank2_b_level,secondary_cleaner.state.floatbank3_a_air,secondary_cleaner.state.floatbank3_a_level,secondary_cleaner.state.floatbank3_b_air,secondary_cleaner.state.floatbank3_b_level,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,127.092003,10.128295,7.25,0.988759,8.547551,10.389648,19.529297,34.174427,14.936526,2.534912,7.476074,2.106679,1549.775757,-498.912140,1551.434204,-516.403442,1549.873901,-498.666595,1554.367432,-493.428131,41885.707031,3481.779053,3520.337158,2.838687,6.100378,2.284912,523.546326,55.486599,36.808594,6.486150,11.986616,6.007990,11.836743,6.005818,11.500771,7.101074,28.029297,19.793808,87.107763,5.008018,0.508728,19.154297,1.170244,999.706909,-404.066986,1603.011353,-434.715027,1602.375000,-442.204468,1598.937256,-451.294128,1404.472046,-455.462982,1416.354980,-451.939636,14.500184,4.694824,8.764648,2.606185,25.853109,-498.526489,23.893660,-501.406281,23.961798,-495.262817,21.940409,-499.340973,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
1,2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,125.629232,10.296251,7.25,1.002663,8.558743,10.

In [19]:
volume_sum_columns = ['rougher.output.concentrate_ag', 'rougher.output.concentrate_pb',
                       'rougher.output.concentrate_sol', 'rougher.output.concentrate_au']

full_df['rougher.output.feed_sum'] = full_df[volume_sum_columns].sum(axis =1)

full_df['rougher.output.feed_au_share'] = full_df['rougher.output.concentrate_au']/full_df['rougher.output.feed_sum']
full_df['rougher.output.feed_ag_share'] = full_df['rougher.output.concentrate_ag']/full_df['rougher.output.feed_sum']
full_df['rougher.output.feed_pb_share'] = full_df['rougher.output.concentrate_pb']/full_df['rougher.output.feed_sum']


volume_sum_columns = ['rougher.input.feed_ag', 'rougher.input.feed_pb', 
                      'rougher.input.feed_sol', 'rougher.input.feed_au']

full_df['rougher.input.feed_sum'] = full_df[volume_sum_columns].sum(axis =1)

full_df['rougher.input.feed_au_share'] = full_df['rougher.input.feed_au']/full_df['rougher.input.feed_sum']
full_df['rougher.input.feed_ag_share'] = full_df['rougher.input.feed_ag']/full_df['rougher.input.feed_sum']
full_df['rougher.input.feed_pb_share'] = full_df['rougher.input.feed_pb']/full_df['rougher.input.feed_sum']

volume_sum_columns = ['rougher.output.tail_ag', 'rougher.output.tail_pb', 
                      'rougher.output.tail_sol', 'rougher.output.tail_au']

full_df['rougher.output.tail_feed_sum'] = full_df[volume_sum_columns].sum(axis =1)

full_df['rougher.output.tail_feed_au_share'] = full_df['rougher.output.tail_au']/full_df['rougher.output.tail_feed_sum']
full_df['rougher.output.tail_feed_ag_share'] = full_df['rougher.output.tail_ag']/full_df['rougher.output.tail_feed_sum']
full_df['rougher.output.tail_feed_pb_share'] = full_df['rougher.output.tail_pb']/full_df['rougher.output.tail_feed_sum']

In [20]:
def prep_df(full_df, l):
    volume_sum_columns = [f'{l}.output.concentrate_ag', f'{l}.output.concentrate_pb',
                        f'{l}.output.concentrate_sol', f'{l}.output.concentrate_au']

    full_df[f'{l}.output.feed_sum'] = full_df[volume_sum_columns].sum(axis =1)

    full_df[f'{l}.output.feed_au_share'] = full_df[f'{l}.output.concentrate_au']/full_df[f'{l}.output.feed_sum']
    full_df[f'{l}.output.feed_ag_share'] = full_df[f'{l}.output.concentrate_ag']/full_df[f'{l}.output.feed_sum']
    full_df[f'{l}.output.feed_pb_share'] = full_df[f'{l}.output.concentrate_pb']/full_df[f'{l}.output.feed_sum']


    volume_sum_columns = [f'{l}.input.feed_ag', f'{l}.input.feed_pb', 
                        f'{l}.input.feed_sol', f'{l}.input.feed_au']

    full_df[f'{l}.input.feed_sum'] = full_df[volume_sum_columns].sum(axis =1)

    full_df[f'{l}.input.feed_au_share'] = full_df[f'{l}.input.feed_au']/full_df[f'{l}.input.feed_sum']
    full_df[f'{l}.input.feed_ag_share'] = full_df[f'{l}.input.feed_ag']/full_df[f'{l}.input.feed_sum']
    full_df[f'{l}.input.feed_pb_share'] = full_df[f'{l}.input.feed_pb']/full_df[f'{l}.input.feed_sum']

    volume_sum_columns = [f'{l}.output.tail_ag', f'{l}.output.tail_pb', 
                        f'{l}.output.tail_sol', f'{l}.output.tail_au']

    full_df[f'{l}.output.tail_feed_sum'] = full_df[volume_sum_columns].sum(axis =1)

    full_df[f'{l}.output.tail_feed_au_share'] = full_df[f'{l}.output.tail_au']/full_df[f'{l}.output.tail_feed_sum']
    full_df[f'{l}.output.tail_feed_ag_share'] = full_df[f'{l}.output.tail_ag']/full_df[f'{l}.output.tail_feed_sum']
    full_df[f'{l}.output.tail_feed_pb_share'] = full_df[f'{l}.output.tail_pb']/full_df[f'{l}.output.tail_feed_sum']

    return full_df

In [21]:
print(full_df.columns)

Index(['date', 'final.output.concentrate_ag', 'final.output.concentrate_pb',
       'final.output.concentrate_sol', 'final.output.concentrate_au',
       'final.output.recovery', 'final.output.tail_ag', 'final.output.tail_pb',
       'final.output.tail_sol', 'final.output.tail_au',
       'primary_cleaner.input.sulfate', 'primary_cleaner.input.depressant',
       'primary_cleaner.input.feed_size', 'primary_cleaner.input.xanthate',
       'primary_cleaner.output.concentrate_ag',
       'primary_cleaner.output.concentrate_pb',
       'primary_cleaner.output.concentrate_sol',
       'primary_cleaner.output.concentrate_au',
       'primary_cleaner.output.tail_ag', 'primary_cleaner.output.tail_pb',
       'primary_cleaner.output.tail_sol', 'primary_cleaner.output.tail_au',
       'primary_cleaner.state.floatbank8_a_air',
       'primary_cleaner.state.floatbank8_a_level',
       'primary_cleaner.state.floatbank8_b_air',
       'primary_cleaner.state.floatbank8_b_level',
       'primary_clean

In [22]:
'primary_cleaner.output.feed_pb_share'.split('.')[0]

'primary_cleaner'

In [66]:
l_cols = {}
for l in ['rougher', 'primary_cleaner',
           'secondary_cleaner', 'final']:
    
    l_cols[l] = {}
    l_cols[l]['input'] = []
    l_cols[l]['output'] = []


    print(l)
    
    for col in full_df.columns:

        #if col.split('.')[0] in ['rougher', 'primary_cleaner',
           #'secondary_cleaner', 'final'] and col.split('.')[1] in ['input', 'ouput']: 
            #l_cols[l][col] = {}


        if l == col.split('.')[0] and col.split('.')[1] == 'input':
            l_cols[l]['input'].append(col)
        elif l == col.split('.')[0] and col.split('.')[1] == 'output':
            l_cols[l]['output'].append(col)
        else:
            continue
        #print(l)

rougher
primary_cleaner
secondary_cleaner
final


In [67]:
l_cols

{'rougher': {'input': ['rougher.input.feed_ag',
   'rougher.input.feed_pb',
   'rougher.input.feed_rate',
   'rougher.input.feed_size',
   'rougher.input.feed_sol',
   'rougher.input.feed_au',
   'rougher.input.floatbank10_sulfate',
   'rougher.input.floatbank10_xanthate',
   'rougher.input.floatbank11_sulfate',
   'rougher.input.floatbank11_xanthate',
   'rougher.input.feed_sum',
   'rougher.input.feed_au_share',
   'rougher.input.feed_ag_share',
   'rougher.input.feed_pb_share'],
  'output': ['rougher.output.concentrate_ag',
   'rougher.output.concentrate_pb',
   'rougher.output.concentrate_sol',
   'rougher.output.concentrate_au',
   'rougher.output.recovery',
   'rougher.output.tail_ag',
   'rougher.output.tail_pb',
   'rougher.output.tail_sol',
   'rougher.output.tail_au',
   'rougher.output.feed_sum',
   'rougher.output.feed_au_share',
   'rougher.output.feed_ag_share',
   'rougher.output.feed_pb_share',
   'rougher.output.tail_feed_sum',
   'rougher.output.tail_feed_au_share',
 

In [78]:
[col for col in full_df.columns if col.split('.')[0] in  ['final']]

['final.output.concentrate_ag',
 'final.output.concentrate_pb',
 'final.output.concentrate_sol',
 'final.output.concentrate_au',
 'final.output.recovery',
 'final.output.tail_ag',
 'final.output.tail_pb',
 'final.output.tail_sol',
 'final.output.tail_au']

In [61]:
for k in l_cols.keys():
    print('#'*100)
    print(k)
    print('#'*100)
    print(l_cols[k])

####################################################################################################
rougher
####################################################################################################
{'input': ['rougher.input.feed_pb_share'], 'output': []}
####################################################################################################
primary_cleaner
####################################################################################################
{'input': [], 'output': []}
####################################################################################################
secondary_cleaner
####################################################################################################
{'input': [], 'output': []}
####################################################################################################
final
####################################################################################################
{'input': [], 'output': []}


In [62]:

for m in ['au', 'ag', 'pb']:
    for l in ['rougher', 'primary_cleaner']:
        full_df =  prep_df(full_df, l)           
        print(m)
        print('input in rougher')
        print(full_df[f'rougher.input.feed_{m}_share'].mean())
        print('output from rougher')
        print(full_df[f'rougher.output.feed_{m}_share'].mean())
        print('output from rougher tails')
        print(full_df[f'rougher.output.tail_feed_{m}_share'].mean())
        print('#'*100)

au
input in rougher
0.145041035962245
output from rougher
0.2908015711661321
output from rougher tails
0.07007107561125496
####################################################################################################


KeyError: "None of [Index(['primary_cleaner.input.feed_ag', 'primary_cleaner.input.feed_pb',\n       'primary_cleaner.input.feed_sol', 'primary_cleaner.input.feed_au'],\n      dtype='object')] are in the [columns]"

In [ ]:
full_df.columns

Index(['date', 'final.output.concentrate_ag', 'final.output.concentrate_pb',
       'final.output.concentrate_sol', 'final.output.concentrate_au',
       'final.output.recovery', 'final.output.tail_ag', 'final.output.tail_pb',
       'final.output.tail_sol', 'final.output.tail_au',
       ...
       'rougher.output.feed_ag_share', 'rougher.output.feed_pb_share',
       'rougher.input.feed_ag_share', 'rougher.input.feed_pb_share',
       'rougher.output.tail_feed_ag_share',
       'rougher.output.tail_feed_pb_share', 'primary_cleaner.output.feed_sum',
       'primary_cleaner.output.feed_au_share',
       'primary_cleaner.output.feed_ag_share',
       'primary_cleaner.output.feed_pb_share'],
      dtype='object', length=104)